In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
import os
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm

# define root directory
ROOT = os.getcwd()
print(ROOT)

dataset_dir = os.path.join(ROOT,"Vision")
scenario_dir = os.path.join(dataset_dir,"Scenario6")
development_dir = os.path.join(scenario_dir, "development_dataset")
challenge_dir = os.path.join(scenario_dir, "challenge_dataset")

/content/drive/MyDrive


In [4]:
import random

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available() :
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [10]:
# load data

import torchvision
import torchvision.transforms as transforms
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms.functional as FT

def transform(image, flag) :
    mean = [0.485, 0.456, 0.406]
    std =  [0.229, 0.224, 0.225]

    if flag == 2 :      #test
        new_image = FT.resize(image,224)
        new_image = FT.to_tensor(new_image)
        new_image = FT.normalize(new_image, mean=mean, std=std)
    else :              #train, val
        new_image = FT.resize(image,224)
        new_image = FT.to_tensor(new_image)
        new_image = FT.normalize(new_image, mean=mean, std=std)

    return new_image

class DeepSense6G(Dataset):
    def __init__(self, data_folder, split, flag):
        self.split = split
        assert self.split in {'Scenario5','Scenario6','Scenario7','Scenario8','Scenario9'}

        self.data_folder = data_folder
        self.scenario_path = os.path.join(self.data_folder, split)
        self.development_path = os.path.join(self.scenario_path, 'development_dataset')
        self.challenge_path = os.path.join(self.scenario_path, 'challenge_dataset')

        self.flag = flag

        self.split = self.split.lower()
        if flag == 0 :      #train
            self.dataframe_path = os.path.join(self.development_path, self.split + '_dev_train.csv')
        elif flag == 1 :    #validation
            self.dataframe_path = os.path.join(self.development_path, self.split + '_dev_val.csv')
        elif flag == 2 :              #test
            self.dataframe_path = os.path.join(self.development_path, self.split + '_dev_test.csv')

        self.dataframe = pd.read_csv(self.dataframe_path)

        self.Images_path = self.dataframe['unit1_rgb_1'].values
        self.Beam_index = self.dataframe['beam_index_1'].values
        self.image_index = self.dataframe['index'].values
        self.Beam_pwr_path = self.dataframe['unit1_pwr_1'].values

    def __getitem__(self, i):

        self.img_rel_path = self.Images_path[i]

        if self.flag == 2 :     #test
            self.img_abs_path = os.path.join(self.development_path, self.img_rel_path)
        else :                  #train, validation
            self.img_abs_path = os.path.join(self.development_path, self.img_rel_path)
        
        image = Image.open(self.img_abs_path, mode='r')
        image = image.convert('RGB')

        beam_index = self.Beam_index[i]

        image = transform(image, self.flag)

        return image, beam_index

    def __len__(self):
        return len(self.Images_path)

    # def _collate_fn(self, batch_size) :

    #     images = list()
    #     beam_indexes = list()

    #     for b in batch_size :
    #         images.append(b[0])
    #         beam_indexes.append(b[1])

    #     images = torch.stack(images, dim=0)

    #     return images, beam_indexes

In [11]:
# device initialize

DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(DEVICE)

cuda


In [12]:
# pre-trained model(ResNet50)

from torchvision import models

resnet50 = models.resnet50(pretrained=True)
num_ftrs = resnet50.fc.in_features
resnet50.fc = torch.nn.Linear(num_ftrs, 64)

model = resnet50.to(DEVICE)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [13]:
# #%pip install torchsummary  #model summary
# from torchsummary import summary 

# def summary_model(model, input_shape=(3,224,224)) :
#     model = model.cuda()
#     summary(model, input_shape)

# print(model)
# summary_model(model)

In [14]:
#training
import time

batch_size = 32
scenario = 'Scenario6'

train_dataset = DeepSense6G(dataset_dir, split=scenario, flag=0)
val_dataset = DeepSense6G(dataset_dir, split=scenario, flag=1)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False) 

epochs = 25
best_loss = 100

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
loss = torch.nn.CrossEntropyLoss().cuda()

train_acc_list = []
val_acc_list = []

for epoch in range(epochs) :
    start = time.time()
    print(f'Epoch {epoch}/{epochs}')

    #train
    train_loss = 0
    correct = 0

    model.train()
    for img, bi in tqdm(train_loader) :

        optimizer.zero_grad()
        y_pred = model(img.cuda())
        cost = loss(y_pred, bi.cuda())

        cost.backward()
        optimizer.step()
        train_loss += cost.item()

        pred = y_pred.data.max(1, keepdim=True)[1]
        correct += pred.cpu().eq(bi.data.view_as(pred)).sum()

    train_loss /= len(train_loader)
    train_acc = correct / len(train_loader.dataset)
    train_acc_list.append(train_acc)

    #validate
    val_loss = 0
    val_correct = 0

    with torch.no_grad() :
        model.eval()
        for img, bi in val_loader :
            y_pred = model(img.cuda())
            cost = loss(y_pred, bi.cuda())
            val_loss += cost.item()

            pred = y_pred.data.max(1, keepdim=True)[1]
            val_correct += pred.cpu().eq(bi.data.view_as(pred)).cpu().sum()
        
        val_loss /= len(val_loader)
        val_acc = val_correct / len(val_loader.dataset)
        val_acc_list.append(val_acc)

        if val_loss < best_loss :
            torch.save({
                'epoch' : epoch,
                'model' : model,
                'model_state_dict' : model.state_dict(),
                'optimizer_state_dict' : optimizer.state_dict(),
                'loss' : cost.item,
            }, os.path.join(dataset_dir, scenario + '/checkpoint/bestCheckpoint.pth'))

            print(f'Epoch {epoch:05d}: val_loss improved from {best_loss:.5f} to {val_loss:.5f}, saving model to bestCheckPoint.pth')
            best_loss = val_loss
        else :
            print(f'Epoch {epoch:05d}: val_loss did not improve')
    print(f'{int(time.time() - start)}s - loss: {train_loss: .5f} - acc: {train_acc:.5f} - val_loss: {val_loss:.5f} - val_acc: {val_acc:.5f}')

Epoch 0/25


100%|██████████| 21/21 [02:06<00:00,  6.00s/it]


Epoch 00000: val_loss improved from 100.00000 to 3.26068, saving model to bestCheckPoint.pth
163s - loss:  3.91291 - acc: 0.10413 - val_loss: 3.26068 - val_acc: 0.31633
Epoch 1/25


100%|██████████| 21/21 [00:26<00:00,  1.25s/it]


Epoch 00001: val_loss improved from 3.26068 to 2.38819, saving model to bestCheckPoint.pth
30s - loss:  2.91780 - acc: 0.34303 - val_loss: 2.38819 - val_acc: 0.36735
Epoch 2/25


...

Epoch 00022: val_loss did not improve
28s - loss:  0.05687 - acc: 0.99387 - val_loss: 2.39977 - val_acc: 0.43878
Epoch 23/25


100%|██████████| 21/21 [00:25<00:00,  1.20s/it]


Epoch 00023: val_loss did not improve
27s - loss:  0.06066 - acc: 0.99541 - val_loss: 2.46967 - val_acc: 0.39796
Epoch 24/25


100%|██████████| 21/21 [00:25<00:00,  1.20s/it]


Epoch 00024: val_loss did not improve
28s - loss:  0.05195 - acc: 0.99234 - val_loss: 2.53076 - val_acc: 0.40816


In [15]:
## 저장되어있는 best checkpoint load

best_model = torch.load(os.path.join(dataset_dir, scenario + '/checkpoint/bestCheckpoint.pth'))['model'] 
best_model.load_state_dict(torch.load(os.path.join(dataset_dir, scenario + '/checkpoint/bestCheckpoint.pth'))['model_state_dict'])  

<All keys matched successfully>

In [21]:
#test 
test_dataset = DeepSense6G(dataset_dir, split=scenario, flag=2)

test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

correct = 0
total = 0

with torch.no_grad() :
    model.eval()
    for img, bi in test_loader :
        pred = model(img.cuda())
        _, predicted = torch.max(pred.data, 1)
        total += bi.size(0)
        correct += (predicted == bi.cuda()).sum().item()

print(scenario,'accuracy:', 100 * correct / total)

Scenario6 accuracy: 52.4390243902439
